In [2]:
import base64

text_to_code = "The Zen of Python, by Tim Peters.\nBeautiful is better than ugly.\nExplicit is better than implicit.\nSimple is better than complex.\nComplex is better than complicated.\nFlat is better than nested.\nSparse is better than dense.\nReadability counts.\nSpecial cases aren't special enough to break the rules.\nAlthough practicality beats purity.\nErrors should never pass silently.\nUnless explicitly silenced.\nIn the face of ambiguity, refuse the temptation to guess.\nThere should be one and preferably only one obvious way to do it.\nAlthough that way may not be obvious at first unless you're Dutch.\nNow is better than never.\nAlthough never is often better than *right* now.\nIf the implementation is hard to explain, it's a bad idea.\nIf the implementation is easy to explain, it may be a good idea.\nNamespaces are one honking great idea let's do more of those!ඞ"

In [3]:
from fractions import Fraction

class SourceStates:
    @staticmethod
    def create_dict(message: str) -> dict:
        states = {}
        for char in message:
            if char not in states:
                states[char] = 1
            else:
                states[char] += 1
        return states

    @staticmethod
    def create_freq_dict(states: dict, message: str) -> dict:
        freq_dict = {}
        total = len(message)
        for char, count in states.items():
            freq_dict[char] = Fraction(count, total)
        return freq_dict

    @staticmethod
    def create_left_borders_dict(freq_dict: dict) -> dict:
        left_borders_dict = {}
        sum_fraction = Fraction(0, 1)
        for char, freq in freq_dict.items():
            left_borders_dict[char] = sum_fraction
            sum_fraction += freq
        return left_borders_dict

    def __init__(self, message):
        self.states = self.create_dict(message)
        self.freq_states = self.create_freq_dict(self.states, message)
        self.left_borders_dict = self.create_left_borders_dict(self.freq_states)


class ArithmeticEncode:
    def __init__(self, message: str):
        self.message = message
        self.left = Fraction(0, 1)
        self.right = Fraction(1, 1)
        self.source_states = SourceStates(self.message)

    def encode(self) -> Fraction:
        for char in self.message:
            left_border = self.source_states.left_borders_dict[char]
            right_border = left_border + self.source_states.freq_states[char]
            current_range = self.right - self.left
            self.right = self.left + current_range * right_border
            self.left = self.left + current_range * left_border

        return (self.left + self.right) / 2

class ArithmeticDecode:
    def __init__(self, coded_val: Fraction, source_states: SourceStates, stop_symbol: str):
        self.stop_symbol = stop_symbol
        self.coded_val = coded_val
        self.source_states = source_states
        self.right = Fraction(1, 1)
        self.left = Fraction(0, 1)

    def decode(self) -> str:
        decoded_message = ''
        left_borders = self.source_states.left_borders_dict
        freq_dict = self.source_states.freq_states
        found_stop_symbol = False
        while not found_stop_symbol:
            current_range = self.right - self.left
            if current_range == 0:
                break
            scaled_value = (self.coded_val - self.left) / current_range
            for char, add_left_freq in left_borders.items():
                add_right_freq = add_left_freq + freq_dict[char]
                if add_left_freq <= scaled_value < add_right_freq:
                    if char == self.stop_symbol:
                        found_stop_symbol = True
                        break
                    decoded_message += char
                    self.right = self.left + current_range * add_right_freq
                    self.left = self.left + current_range * add_left_freq
        return decoded_message

code = ArithmeticEncode(text_to_code)
encoded_value = code.encode()
decode = ArithmeticDecode(encoded_value, SourceStates(text_to_code), "ඞ")
print(f"Frequencies: {code.source_states.freq_states}")
print(f"Encoded: {encoded_value}")
print(f"Decoded: {decode.decode()}")

Frequencies: {'T': Fraction(3, 850), 'h': Fraction(31, 850), 'e': Fraction(9, 85), ' ': Fraction(123, 850), 'Z': Fraction(1, 850), 'n': Fraction(4, 85), 'o': Fraction(43, 850), 'f': Fraction(11, 850), 'P': Fraction(1, 425), 'y': Fraction(1, 50), 't': Fraction(38, 425), ',': Fraction(2, 425), 'b': Fraction(2, 85), 'i': Fraction(1, 17), 'm': Fraction(8, 425), 'r': Fraction(16, 425), 's': Fraction(43, 850), '.': Fraction(19, 850), '\n': Fraction(19, 850), 'B': Fraction(1, 850), 'a': Fraction(1, 17), 'u': Fraction(2, 85), 'l': Fraction(33, 850), 'g': Fraction(11, 850), 'E': Fraction(1, 425), 'x': Fraction(3, 425), 'p': Fraction(2, 85), 'c': Fraction(8, 425), 'S': Fraction(3, 850), 'C': Fraction(1, 850), 'd': Fraction(8, 425), 'F': Fraction(1, 850), 'R': Fraction(1, 850), "'": Fraction(2, 425), 'k': Fraction(1, 425), 'A': Fraction(3, 850), 'v': Fraction(1, 170), 'U': Fraction(1, 850), 'I': Fraction(3, 850), 'w': Fraction(2, 425), 'D': Fraction(1, 850), 'N': Fraction(1, 425), '*': Fraction(1

In [4]:
import heapq
from collections import Counter

class Node:

    def __init__(self, char: str, freq: float):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

def build_huffman_tree(frequencies: dict) -> Node:
    heap = [Node(char, freq) for char, freq in frequencies.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = Node(None, node1.freq + node2.freq)
        merged.left = node1
        merged.right = node2
        heapq.heappush(heap, merged)
    return heap[0]

def generate_huffman_codes(node, prefix: str='', huffman_codes: dict = {}) -> dict:
    if node is None:
        return
    if node.char is not None:
        huffman_codes[node.char] = prefix
    else:
        generate_huffman_codes(node.left, prefix + '0', huffman_codes)
        generate_huffman_codes(node.right, prefix + '1', huffman_codes)
    return huffman_codes

def huffman_encoding(message: str) -> tuple:
    frequencies = Counter(message)
    root = build_huffman_tree(frequencies)
    huffman_codes = generate_huffman_codes(root)
    encoded_text = ''.join([huffman_codes[char] for char in message])
    return encoded_text, huffman_codes

def huffman_decoding(encoded_text: str, huffman_codes: dict) -> str:
    reverse_huffman_codes = {code: char for char, code in huffman_codes.items()}
    decoded_message = ""
    current_code = ""
    for bit in encoded_text:
        current_code += bit
        if current_code in reverse_huffman_codes:
            decoded_message += reverse_huffman_codes[current_code]
            current_code = ""
    return decoded_message

message = text_to_code
encoded_text, huffman_codes = huffman_encoding(message)
decoded_text = huffman_decoding(encoded_text, huffman_codes)
print(f"{message}")
print(f"Characters' codes: {huffman_codes}")
print(f"Encoded: {encoded_text}")
print(f"Decoded: {decoded_text}")

The Zen of Python, by Tim Peters.
Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one and preferably only one obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea let's do more of those!ඞ
Characters' codes: {'n': '0000', 'b': '00010', 'u': '00011', 'p': '00100', 'v': '0010100', 'F': '001010100', 'E': '001010101', 'I': '00101011', 'g': 

In [27]:
class Base64:
    @staticmethod
    def char_to_binary(char: str) -> str:
        return ''.join(format(ord(char), '08b'))
    @staticmethod
    def split_string_by_chunks(s: str, n: int) -> list:
        return [s[i:i + n] for i in range(0, len(s), n)]
    
    base64chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    code_to_base64char = {format(i, '06b'): char for i, char in enumerate(base64chars)}
    @staticmethod
    def base64char_to_code(char: str) -> str:
        if char == "=":
            return ""
        base64char_to_code = {code: char for char, code in Base64.code_to_base64char.items()}
        return base64char_to_code[char]
    



In [28]:
class Base64Encoder(Base64):
    @staticmethod
    def data_to_utf8_stream(data: str) -> str:
        stream = ""
        for char in data:
            stream += Base64Encoder.char_to_binary(char)
        return stream
    @staticmethod
    def stream_to_base64(stream: str) -> str:
        stream_chunks = Base64Encoder.split_string_by_chunks(stream, 6)
        while len(stream_chunks[len(stream_chunks) - 1]) < 6:
            stream_chunks[len(stream_chunks) - 1] += '0'
        base64_stream = ""
        for code in stream_chunks:
            base64_stream += Base64Encoder.code_to_base64char[code]
        return base64_stream
    def __init__(self, data: str):
        self.data = data
    def encode(self) -> str:
        stream = Base64Encoder.data_to_utf8_stream(self.data)
        base64_string = Base64Encoder.stream_to_base64(stream)
        while len(base64_string) % 4 != 0:
            base64_string += "="
        return base64_string


In [29]:
class Base64Decoder(Base64):
    @staticmethod
    def base64_to_base64_stream(data: str) -> str:
        base64_stream = ""
        for char in data:
            coded_char = Base64Decoder.base64char_to_code(char)
            base64_stream = base64_stream + coded_char
        return base64_stream
    @staticmethod
    def b64stream_to_utf8(data: str) -> str:
        base64_stream_chunks = Base64Decoder.split_string_by_chunks(data, 8)
        utf8_string = ""
        for code in base64_stream_chunks:
            decimal_value = int(code, 2)
            utf8_string += chr(decimal_value)
        return utf8_string
    def __init__(self, data: str):
        self.data = data
    def decode(self) -> str:
        b64_stream = Base64Decoder.base64_to_base64_stream(self.data)
        utf8_string = Base64Decoder.b64stream_to_utf8(b64_stream)
        return utf8_string
        

In [30]:
encoder = Base64Encoder(text_to_code)
encoded_string = encoder.encode()
print(f"Encoded: {encoded_string}")

Encoded: VGhlIFplbiBvZiBQeXRob24sIGJ5IFRpbSBQZXRlcnMuCkJlYXV0aWZ1bCBpcyBiZXR0ZXIgdGhhbiB1Z2x5LgpFeHBsaWNpdCBpcyBiZXR0ZXIgdGhhbiBpbXBsaWNpdC4KU2ltcGxlIGlzIGJldHRlciB0aGFuIGNvbXBsZXguCkNvbXBsZXggaXMgYmV0dGVyIHRoYW4gY29tcGxpY2F0ZWQuCkZsYXQgaXMgYmV0dGVyIHRoYW4gbmVzdGVkLgpTcGFyc2UgaXMgYmV0dGVyIHRoYW4gZGVuc2UuClJlYWRhYmlsaXR5IGNvdW50cy4KU3BlY2lhbCBjYXNlcyBhcmVuJ3Qgc3BlY2lhbCBlbm91Z2ggdG8gYnJlYWsgdGhlIHJ1bGVzLgpBbHRob3VnaCBwcmFjdGljYWxpdHkgYmVhdHMgcHVyaXR5LgpFcnJvcnMgc2hvdWxkIG5ldmVyIHBhc3Mgc2lsZW50bHkuClVubGVzcyBleHBsaWNpdGx5IHNpbGVuY2VkLgpJbiB0aGUgZmFjZSBvZiBhbWJpZ3VpdHksIHJlZnVzZSB0aGUgdGVtcHRhdGlvbiB0byBndWVzcy4KVGhlcmUgc2hvdWxkIGJlIG9uZSBhbmQgcHJlZmVyYWJseSBvbmx5IG9uZSBvYnZpb3VzIHdheSB0byBkbyBpdC4KQWx0aG91Z2ggdGhhdCB3YXkgbWF5IG5vdCBiZSBvYnZpb3VzIGF0IGZpcnN0IHVubGVzcyB5b3UncmUgRHV0Y2guCk5vdyBpcyBiZXR0ZXIgdGhhbiBuZXZlci4KQWx0aG91Z2ggbmV2ZXIgaXMgb2Z0ZW4gYmV0dGVyIHRoYW4gKnJpZ2h0KiBub3cuCklmIHRoZSBpbXBsZW1lbnRhdGlvbiBpcyBoYXJkIHRvIGV4cGxhaW4sIGl0J3MgYSBiYWQgaWRlYS4KSWYgdGhlIGltcGxlbWVudGF0aW9

In [31]:
decoder = Base64Decoder(encoded_string)
decoded_string = decoder.decode()
print(f"Decoded: {decoded_string}")

Decoded: The Zen of Python, by Tim Peters.
Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one and preferably only one obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea let's do more of those!Ù


In [1]:
from coding_evaluation import CodingEvaluator

def base64_evaluation(data: str, is_encoding: bool, to_binary: bool) -> str:
    encoder = Base64Encoder(data)
    encoded_string = encoder.encode()
    decoder = Base64Decoder(data)
    decoded_string = decoder.decode()
    if is_encoding:
        return encoded_string
    else:
        return decoded_string

def get_test_cases() -> list:
    file = open(r"../cache/lab3/test_cases.txt", "r")
    test_cases = file.read().split(";")
    return test_cases

evaluator = CodingEvaluator(get_test_cases=get_test_cases, base64=base64_evaluation)
evaluator.evaluate()
    

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 100: character maps to <undefined>